Simple script to fetch all the latest measurement probe data in Africa, create seperate csv files per country, and cocatenate into a single csv file.

***This notebook, and csv files must be in the same directory***

***Delete all the csv files in the directory if you need to run this script again***

In [1]:
import requests
import json
from pandas.io.json import json_normalize 
import time
from datetime import date
import pandas as pd
import _thread
import matplotlib.pyplot as plt
import matplotlib.dates
from datetime import datetime
import numpy as np
import csv
import os
import glob
os.chdir('/home/tunde/RESEARCH_&_INNOVATION/RIPE_Atlas_Analysis/All_Countries/')

%matplotlib inline

#set ggplot style
plt.style.use('default')
plt.rcParams.update({'font.size': 14})

ATLAS_PROBES_URI = "https://stat.ripe.net/data/atlas-probes/data.json?resource="
ATLAS_DEPLOYMENT_URI = "https://stat.ripe.net/data/atlas-probe-deployment/data.json?resource="

ccs = [
'EG', 'BJ', 'CI', 'CV', 'GH', 'GM', 'GN', 'GW', 'AO', 'CF', 'CG', 'CM', 'GA', 'GQ', 'TD', 'BI', 'DJ', 'ER', 'ET', 'KM', 'BW', 'MA', 'SD', 'TN', 'LR', 'ML', 'MR', 'NE', 'NG', 'SL', 'SN', 'TG', 'ST', 'KE', 'MG', 'MU', 'MW', 'MZ', 'RE', 'RW', 'SC', 'SO', 'UG', 'LS', 'NA', 'SZ', 'ZA', 'DZ', 'EH', 'LY', 'BF', 'SH', 'CD', 'TZ', 'YT', 'ZM', 'ZW']

Function to get probe status per country and create a seperate csv file per country in the list ***ccs*** above

In [2]:
def getDeployment(cc):
    
    test_url = ATLAS_DEPLOYMENT_URI + cc
    
    df = pd.DataFrame()
    
    try:
        r = requests.get(test_url)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    try:
        data = json.loads(r.text)['data']['deployments'][0]['deployment']
        df = json_normalize(data)
        df['cc'] = cc
    except KeyError as e:
        pass
    
    return df

Loop through all the created csv files in the present directory the notebook is

In [13]:
for cc in ccs:
        print(cc)
        a = getDeployment(cc)
        a.to_csv(cc +'latest_deployments.csv')

EG
BJ
CI
CV
GH
GM
GN
GW
AO
CF
CG
CM
GA
GQ
TD
BI
DJ
ER
ET
KM
BW
MA
SD
TN
LR
ML
MR
NE
NG
SL
SN
TG
ST
KE
MG
MU
MW
MZ
RE
RW
SC
SO
UG
LS
NA
SZ
ZA
DZ
EH
LY
BF
SH
CD
TZ
YT
ZM
ZW


 Merge the csv files created above 

In [14]:
extension = 'csv'

all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv
combined_csv.to_csv( "2020_africa_probes_deployment.csv", index=False, encoding='utf-8-sig')

print('csv files successfully merged')

csv files successfully merged


Confirm that the csv files were successfully merged


In [15]:
df = pd.read_csv('/home/tunde/RESEARCH_&_INNOVATION/RIPE_Atlas_Analysis/All_Countries/2020_africa_probes_deployment.csv')
df = df.iloc[:,[0,1,2,3,4,5,6]]
df.head(20)
df.shape


(109923, 7)

Rename the columns and confirm

In [16]:
df = pd.read_csv('/home/tunde/RESEARCH_&_INNOVATION/RIPE_Atlas_Analysis/latest_deployments.csv')
df.rename(columns = {'statuses.abandoned': 'abandoned', 'statuses.connected': 'connected', 'statuses.disconnected': 'disconnected', 'statuses.neverseen': 'neverseen'}, inplace = True)
df = df.iloc[:,[0,1,2,3,4,5,]]
df.head(20)

,Unnamed: 0,date,abandoned,connected,disconnected,neverseen
0,0,2020-09-21,18,3,1,0.0
1,1,2020-09-20,18,3,1,0.0
2,2,2020-09-19,18,3,1,0.0
3,3,2020-09-18,18,2,2,0.0
4,4,2020-09-17,18,2,2,0.0
5,5,2020-09-16,18,3,1,0.0
6,6,2020-09-15,18,3,1,0.0
7,7,2020-09-14,18,2,2,0.0
8,8,2020-09-13,18,3,1,0.0
9,9,2020-09-12,18,3,1,0.0


In [ ]:
df_agg = df.groupby(['date'])['neverseen', 'abandoned','connected','disconnected'].sum().reset_index()
#df_agg.set_index('date')
df_agg.head()

In [ ]:
df_neverseen = df.iloc[:,[0,4]]
df_neverseen.columns = {'date','amount'}
df_neverseen= df_neverseen.assign(type = 'neverseen')

df_abandoned = df.iloc[:,[0,1]]
df_abandoned.columns = {'date','amount'}
df_abandoned = df_abandoned.assign(type = 'abandoned')

df_connected = df.iloc[:,[0,2]]
df_connected.columns = {'date','amount'}
df_connected = df_connected.assign(type = 'connected')

df_disconnected = df.iloc[:,[0,3]]
df_disconnected.columns = {'date','amount'}
df_disconnected = df_disconnected.assign(type = 'disconnected')

df_agg2 = pd.concat([df_neverseen, df_abandoned, df_connected, df_disconnected])
df_agg2.head(10)